## Overview





Today we examine a few important ideas in data science and machine learning. They are:

1.  Data preprocessing. We mostly focus on how to scale data so that each feature has about the same scale.
2.  How to combine many data processing/fitting steps into a pipeline that is easy to work with
3.  How to use k-fold validation to make sure your model fit doesn't depend too much on one data set sample.
4.  Automating the optimization of model parameters using cross-validation.





## Data preprocessing





When your features have varying orders of magnitude, it is often a good idea to standardize them. That makes each feature have a similar importance. Some algorithms even expect that the input features have been standardized (e.g. PCA, and many kinds of regularization). The two most common approaches are:

-   **zero-mean, unit variance:** You subtract the mean of each column, and normalize by the variance of the column
-   **min-max:** you scale the column so that the minimum value maps to a constant (often 0 or -1), and the maximum value maps to a constant (often 1).

`sklearn` provides a [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module to make this "simple".

We will explore this with this data set which represents the energy of a Cu dimer as a function of the distance between the two atoms. Our goal is to develop a model that fits this. We know some physics about this system, and that is as the atoms approach each other, they should strongly repel each other, and the energy would go to infinity, and as the atoms get far apart, they cannot interact, so the energy should go to a constant. You might think the energy should go to zero,, but this depends on what reference system the energy zero is defined for. The zero in this system is defined as the minimum energy for *bulk* Cu in an fcc unit cell. So, this system levels out at an energy that is much higher than that.





In [ ]:
import json

with open('data.json') as f:
    data = json.load(f)

import matplotlib.pyplot as plt
plt.plot(data['distance'], data['energy'], 'bo')
plt.xlabel('distance')
plt.ylabel('energy');



First, let's make some columns. Instead of manually creating these, let's leverage the capability of `sklearn` to build the features for us. Here is an example of a second order polynomial.





In [ ]:
import numpy as np

X = np.array([data['distance']]).T

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
poly.fit_transform(X)



You can see that there are some orders of magnitude variation across the columns. We can use standard methods in sklearn to scale these features. This is the *zero-mean and unit-variance* approach.





In [ ]:
from sklearn import preprocessing

X_scaled = preprocessing.scale(poly.fit_transform(X))
X_scaled



We can check that we get what we asked for:





In [ ]:
X_scaled.mean(), X_scaled.std(axis=0)



We can also transform to min-max this way, the default here is to scale between 0 and 1.





In [ ]:
minmax = preprocessing.MinMaxScaler()
minmax.fit_transform(poly.fit_transform(X))



There are other ways to do the scaling, and sometimes you have to be careful that you don't do bad things. For example if you have very sparse data, it may not make sense to center it.





### Pipeline





These steps can be combined in what is called a `pipeline`. The idea is that data goes into the pipe and is transformed in a series of steps, and the results come out the end of the pipe. The code above can be condensed into this concept like this. Here we also perform a linear regression.





In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

poly = PolynomialFeatures(4)
scaler = preprocessing.StandardScaler()
model = linear_model.LinearRegression()

X = np.array([data['distance']]).T
y = np.array([data['energy']]).T

pipe = Pipeline([('poly', poly),
                 ('scale', scaler),
                 ('linregress', model)])

pipe.fit(X, y)



Here, we can use visualization to see how well the fit works with this data.





In [ ]:
dfit = np.linspace(1.5, 5)

plt.plot(data['distance'], data['energy'], 'bo',
         dfit, pipe.predict(dfit[:, None]));



The model sort of gets the right idea, but you can see there are issues with it. The local maximum near 3.7 is not real, and the increase in energy past d=4.5 is also not physically correct. We can work with different polynomial orders to see if that is fixable. The polynomial order is considered a *hyperparameter* here.





## k-fold validation / cross-validation





[https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html)

We have 75 data points, and the best thing to do in training would be to split the data into train and test sets. Last time we saw a way to that in `sklearn`. The problem with doing it once is that the results may depend on the specific set of data, and it would be nice to see several trials to make sure it doesn't. You can code this yourself, but here is a better, more formalized way to do it. The idea is called `K-fold` validation, where you split the data into `k` folds that are splits into training and testing data. Then, we fit on k-1 of the folds, and test on the one left out. We do that for all the combinations of folds.





In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

Xfit = np.linspace(1.8, 4.8)[:, None]
plt.plot(X, y, 'bo')

for train, test in kf.split(X):
    pipe.fit(X[train], y[train])
    plt.plot(Xfit, pipe.predict(Xfit))
    plt.plot(X[test], pipe.predict(X[test]), 'ro')
    print(pipe.score(X[train], y[train]), pipe.score(X[test], y[test]))



You can see here that the results depend on the specific data set that is chosen. In every case, the score of the train data is ok, but for some models the test data is not as good. This is commonly observed, and it indicates over-fitting in the train data, and poor generalization in the test data.

A common metric for judging the fit is called the cross-validation score. This is obtained by averaging the score across all the folds. We would like this score to be a minimum for our model.





In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipe, X, y, cv=5)
print(scores)
print(f"Accuracy: {scores.mean():0.2f} (+/- {scores.std() * 2:0.2f})")



## hyperparameter optimization





The next goal we have is to figure out what the best polynomial to use might be. We will leverage `sklearn` again to do this search for us. First, we get a list of parameters that we can change in our `pipe`.





In [ ]:
pipe.get_params().keys()



We want to vary the `poly_degree` parameter to find the one with the best CV score. We make a list of integer values this argument can take.





In [ ]:
pdegree = np.arange(4, 15)
param_grid = [{'poly__degree': pdegree}]

from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(pipe, param_grid)
clf.fit(X[train], y[train])

print(clf.best_params_)



That tells us an 11th order polynomial is best here. We get a lot of data from this.





In [ ]:
clf.cv_results_



The order 11 is the one that minimizes the average test score *and* std, and is the best compromise on these.





In [ ]:
plt.plot(pdegree, clf.cv_results_['mean_test_score'],
         pdegree, clf.cv_results_['std_test_score'])
plt.ylim([-100, 100])
plt.legend(['mean', 'std'])
plt.xlabel('Polynomial order')
plt.ylabel('CV score')



We can also see here that this is a good compromise. Note, however, we still cannot reliably extrapolate with this model because it is still just a polynomial model, and it does not contain any physics.





In [ ]:
Xfit = np.linspace(1.5, 5.5)[:, None]
plt.plot(X, y, 'bo')
plt.plot(Xfit, clf.predict(Xfit), 'r-')
plt.xlabel('Distance')
plt.ylabel('energy')
plt.legend(['data', 'fit']);



**Exercises** Read more about these sklearn functions. How would you plot the fit for each polynomial degree?





### Summary





We illustrated how to leverage `sklearn` to do a single hyperparameter optimization here, the polynomial order. Typical machine learning methods have many hyperparameters. We could add new ones here if you include regularization, e.g Lasso, or Ridge would add one more hyperparameter, and Elastic net would add two. Neural networks have hyperparameters that include how many hidden layers, how many neurons per layer, and which activation functions to use. As the number of parameters grow, it is increasingly important to have automated methods for exploring and optimizing them, as well as principled methods for deciding which models are better.



